In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add, Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, Concatenate, Reshape, Activation, BatchNormalization, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.nn import local_response_normalization
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.keras.activations import relu
import sys
import matplotlib.pyplot as plt
import math
import matplotlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l2, l1, l1_l2
from itertools import permutations, combinations
import cv2

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
# config.gpu_options.per_process_gpu_memory_fraction = 0.60
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 3080, pci bus id: 0000:41:00.0, compute capability: 8.6



In [3]:
#### Using CIFAR10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255


y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10) 

global_batch_size = 32
image_resize = 32

########## Train
#### removing data augmentation only here for profiling
train_datagen = ImageDataGenerator(rescale=1./255)

# train_datagen = ImageDataGenerator(rescale=1./255, featurewise_center = True,
#                                    rotation_range = 20, horizontal_flip = True, height_shift_range = 0.2,
#                                    width_shift_range = 0.2, zoom_range = 0.2, channel_shift_range = 0.2)


train_it = train_datagen.flow_from_directory(
        'cifar10/train',
        class_mode='categorical',
        target_size=(image_resize, image_resize),
        batch_size=global_batch_size)


############ Test
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

validation_it = test_datagen.flow_from_directory(
        'cifar10/test',
        class_mode='categorical',
        target_size=(image_resize, image_resize),
        batch_size=global_batch_size,
        subset = "training",seed = 545)

test_it = test_datagen.flow_from_directory(
        'cifar10/test',
        class_mode='categorical',
        target_size=(image_resize, image_resize),
        batch_size=global_batch_size,
        subset = "validation",
        seed = 545)


for data_batch, labels_batch in train_it:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break


Found 50000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
data batch shape: (32, 32, 32, 3)
labels batch shape: (32, 10)


In [4]:
class CustomLoss(keras.losses.Loss):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor

    def call(self, y_true, y_pred):
        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        return ce * self.factor
    
class Residual_block(tf.keras.Model):
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        
        self.conv1 = Conv2D(num_channels, kernel_size=3, padding='same', strides=strides, kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001))
        self.conv2 = Conv2D(num_channels, kernel_size=3, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001))
        self.conv3 = None
        
        if use_1x1conv:
            self.conv3 = Conv2D(num_channels, kernel_size=1, strides=strides, kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001))
            
        self.bn1 = BatchNormalization()
        self.bn2 = BatchNormalization()
        
    def call(self, X, training):
        
        if (training != False and training != True):
            training = False
        
        Y = self.conv1(X)
        Y = self.bn1 (Y, training = training)
        Y = relu(Y)
        Y = self.conv2(Y)
        Y = self.bn2 (Y, training = training)
        
        if self.conv3 is not None:
            X = self.conv3(X)
            
        Y += X
        Y = relu(Y)
        return Y


class CIFAR_ResNet(tf.keras.Model):
    def __init__(self, model_n, branch_number):
        super(CIFAR_ResNet, self).__init__()
        
        self.model_n = model_n
        self.branch_number = branch_number
        self.residual_layers = []
        
        ######## Begining Layers
        self.conv1 = Conv2D(16, kernel_size=3, padding='same',kernel_initializer='he_uniform', input_shape=(image_resize, image_resize, 3), kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001))
        self.bn1 = BatchNormalization()
        
        
        ######## ResNet layers
        ### first block, 16 channels
        for i in range(self.model_n):
            self.residual_layers.append(Residual_block(16))
            
        ### second block, 32 channels
        for i in range(self.model_n):
            if i == 0:
                self.residual_layers.append(Residual_block(32, use_1x1conv=True, strides=2))
            else:
                self.residual_layers.append(Residual_block(32))
                
        ### third block, 64 channels
        self.residual_layers_block3 = []
        for i in range(self.model_n):
            if i == 0:
                self.residual_layers.append(Residual_block(64, use_1x1conv=True, strides=2))
            else:
                self.residual_layers.append(Residual_block(64))

        
        ######## OUTPUT Layers
        self.pool_out = GlobalAveragePooling2D()
        self.flat_out = Flatten()

        self.dense_layers = []
        for i in range(self.branch_number):
            self.dense_layers.append(Dense(10, kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001), activation='softmax'))
        

    def call(self, inputs, training):
        inference_flag = -1000
        layer_ind_counter = 0
        comp_latency_list_backbone = [0 for i in range(self.branch_number)]
        comp_latency_list_exitbranch = [0 for i in range(self.branch_number)]
        out_vector_list = [[] for i in range(self.branch_number)]
        
        if (training != False and training != True):
            inference_flag = 1000
            training = False
            
            
        ##### Begining layers
        start_time = tf.timestamp()
        x = self.conv1(inputs)
        x = self.bn1 (x, training = training)
        x = relu (x)
        comp_latency_list_backbone[layer_ind_counter] = tf.timestamp() - start_time
        
        
        # exit branch
        start_time = tf.timestamp()
        temp = self.pool_out(x)
        temp = self.flat_out(temp)
        out_vector_list[layer_ind_counter] = self.dense_layers[layer_ind_counter](temp)
        comp_latency_list_exitbranch[layer_ind_counter] = tf.timestamp() - start_time
        
        
        
        ##### ResNet Layers
        for layer in self.residual_layers:
            layer_ind_counter += 1
            start_time = tf.timestamp()
            x = layer (x)
            comp_latency_list_backbone[layer_ind_counter] = tf.timestamp() - start_time
            
            # add exit branch after each resnet block
            start_time = tf.timestamp()
            temp = self.pool_out(x)
            temp = self.flat_out(temp)
            out_vector_list[layer_ind_counter] = self.dense_layers[layer_ind_counter](temp)
            comp_latency_list_exitbranch[layer_ind_counter] = tf.timestamp() - start_time
            


        if (inference_flag == 1000):
            return(out_vector_list, comp_latency_list_backbone, comp_latency_list_exitbranch)
        
        
        return out_vector_list

In [5]:
##### ResNet20 + CIFAR10(32x32)

##### load weight
opt = tf.keras.optimizers.SGD(momentum=0.9)

##### for ResNet20
branch_number= 10
model_n = 3

ls = [CustomLoss(1) for i in range(branch_number)]

model_CIFAR_ResNet = CIFAR_ResNet(model_n,branch_number)
model_CIFAR_ResNet.compile(optimizer=opt, loss=ls, metrics=['accuracy'])
model_CIFAR_ResNet.load_weights('profiling_models/ResNet20_10out_bs32_epoch800_lr0.01_new')
model_CIFAR_ResNet.evaluate(test_it);

157/157 [==============================] - 5s 19ms/step - loss: 8.5724 - output_1_loss: 1.7556 - output_2_loss: 1.2076 - output_3_loss: 1.0540 - output_4_loss: 1.0171 - output_5_loss: 0.7794 - output_6_loss: 0.5720 - output_7_loss: 0.5007 - output_8_loss: 0.4383 - output_9_loss: 0.3808 - output_10_loss: 0.3652 - output_1_accuracy: 0.3992 - output_2_accuracy: 0.5918 - output_3_accuracy: 0.6510 - output_4_accuracy: 0.6742 - output_5_accuracy: 0.7652 - output_6_accuracy: 0.8232 - output_7_accuracy: 0.8432 - output_8_accuracy: 0.8662 - output_9_accuracy: 0.8790 - output_10_accuracy: 0.8848


In [5]:
##### ResNet20 + CIFAR10(128x128)

##### load weight
opt = tf.keras.optimizers.SGD(momentum=0.9)

##### for ResNet20
branch_number= 10
model_n = 3

ls = [CustomLoss(1) for i in range(branch_number)]

model_CIFAR_ResNet = CIFAR_ResNet(model_n,branch_number)
model_CIFAR_ResNet.compile(optimizer=opt, loss=ls, metrics=['accuracy'])
model_CIFAR_ResNet.load_weights('profiling_models/ResNet20_10out_bs32_epoch500_lr0.01_128x128')
model_CIFAR_ResNet.evaluate(test_it);

157/157 [==============================] - 6s 23ms/step - loss: 9.8817 - output_1_loss: 1.6978 - output_2_loss: 1.3324 - output_3_loss: 1.2345 - output_4_loss: 1.1332 - output_5_loss: 1.0083 - output_6_loss: 0.8086 - output_7_loss: 0.7023 - output_8_loss: 0.5606 - output_9_loss: 0.4608 - output_10_loss: 0.4325 - output_1_accuracy: 0.3832 - output_2_accuracy: 0.5372 - output_3_accuracy: 0.5820 - output_4_accuracy: 0.6084 - output_5_accuracy: 0.6640 - output_6_accuracy: 0.7296 - output_7_accuracy: 0.7618 - output_8_accuracy: 0.8222 - output_9_accuracy: 0.8596 - output_10_accuracy: 0.8668


In [5]:
##### ResNet44 + CIFAR10(32x32)

##### load weight
opt = tf.keras.optimizers.SGD(momentum=0.9)

##### for ResNet44
branch_number= 22
model_n = 7

ls = [CustomLoss(1) for i in range(branch_number)]

model_CIFAR_ResNet = CIFAR_ResNet(model_n,branch_number)
model_CIFAR_ResNet.compile(optimizer=opt, loss=ls, metrics=['accuracy'])
model_CIFAR_ResNet.load_weights('profiling_models/ResNet44_22out_bs32_epoch500_lr0.01')
model_CIFAR_ResNet.evaluate(test_it);

157/157 [==============================] - 12s 45ms/step - loss: 16.0903 - output_1_loss: 1.7119 - output_2_loss: 1.4054 - output_3_loss: 1.2121 - output_4_loss: 1.0742 - output_5_loss: 0.9618 - output_6_loss: 0.8778 - output_7_loss: 0.7946 - output_8_loss: 0.8157 - output_9_loss: 0.7359 - output_10_loss: 0.5949 - output_11_loss: 0.5141 - output_12_loss: 0.4587 - output_13_loss: 0.4226 - output_14_loss: 0.4055 - output_15_loss: 0.4039 - output_16_loss: 0.3943 - output_17_loss: 0.3567 - output_18_loss: 0.3354 - output_19_loss: 0.3271 - output_20_loss: 0.3258 - output_21_loss: 0.3224 - output_22_loss: 0.3224 - output_1_accuracy: 0.3992 - output_2_accuracy: 0.5186 - output_3_accuracy: 0.5884 - output_4_accuracy: 0.6542 - output_5_accuracy: 0.6904 - output_6_accuracy: 0.7264 - output_7_accuracy: 0.7536 - output_8_accuracy: 0.7466 - output_9_accuracy: 0.7704 - output_10_accuracy: 0.8142 - output_11_accuracy: 0.8436 - output_12_accuracy: 0.8582 - output_13_accuracy: 0.8702 - output_14_accura

In [5]:
##### ResNet110 + CIFAR10(32x32)

##### load weight
opt = tf.keras.optimizers.SGD(momentum=0.9)

##### for ResNet110
branch_number= 55
model_n = 18

ls = [CustomLoss(1) for i in range(branch_number)]

model_CIFAR_ResNet = CIFAR_ResNet(model_n,branch_number)
model_CIFAR_ResNet.compile(optimizer=opt, loss=ls, metrics=['accuracy'])
model_CIFAR_ResNet.load_weights('profiling_models/ResNet110_55out_bs32_epoch800_lr0.01')
model_CIFAR_ResNet.evaluate(test_it);

157/157 [==============================] - 32s 82ms/step - loss: 35.5343 - output_1_loss: 1.8200 - output_2_loss: 1.5343 - output_3_loss: 1.5059 - output_4_loss: 1.3007 - output_5_loss: 1.1630 - output_6_loss: 1.0567 - output_7_loss: 1.0033 - output_8_loss: 0.9489 - output_9_loss: 0.9098 - output_10_loss: 0.8414 - output_11_loss: 0.8184 - output_12_loss: 0.7825 - output_13_loss: 0.7167 - output_14_loss: 0.6884 - output_15_loss: 0.6591 - output_16_loss: 0.6486 - output_17_loss: 0.6587 - output_18_loss: 0.6652 - output_19_loss: 0.7007 - output_20_loss: 0.6712 - output_21_loss: 0.6390 - output_22_loss: 0.5755 - output_23_loss: 0.5093 - output_24_loss: 0.4625 - output_25_loss: 0.4157 - output_26_loss: 0.3866 - output_27_loss: 0.3581 - output_28_loss: 0.3449 - output_29_loss: 0.3389 - output_30_loss: 0.3296 - output_31_loss: 0.3224 - output_32_loss: 0.3201 - output_33_loss: 0.3181 - output_34_loss: 0.3149 - output_35_loss: 0.3146 - output_36_loss: 0.3146 - output_37_loss: 0.3206 - output_38

In [6]:
###### now the profiling part
###### TRAIN PART

In [7]:
##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches
print("%%%%%%%%%%%%%%%%%%%%%%% ResNet20 + CIFAR10(32x32)%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
per_sample_label_list = []
per_sample_out_vector_list = []
per_sample_comp_latency_backbone_list = []
per_sample_comp_latency_exitbranch_list = []


#### saving intermediate data once
for i in range (len(train_it)):
    temp_batch = train_it[i]

    for j in range (len(temp_batch[0])):
        pic = temp_batch[0][j]
        label = temp_batch[1][j]
        per_sample_label_list.append(np.array(label).reshape(1,10))

        res = model_CIFAR_ResNet(np.array(pic.reshape(1,image_resize,image_resize,3)), training = 1000)
   
        per_sample_out_vector_list.append(res[0])
        per_sample_comp_latency_backbone_list.append(res[1])
        per_sample_comp_latency_exitbranch_list.append(res[2])

print ("computation latencies backbone(train data)   ", np.mean(per_sample_comp_latency_backbone_list, axis=0)*1000)
print ("computation latencies exitbranch(train data)   ", np.mean(per_sample_comp_latency_exitbranch_list, axis=0)*1000)




##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches

entropy_threshold_list = np.linspace(0.000000001, 0.999999, num=20)

### all possible branches, and chosen number
chosen_number = branch_number
I = list(range(1, branch_number+1))

#### number of train samples for simulation
sample_number = train_it.samples



#### per placement, in this case it is only 1, all the possible branches
for item in combinations(I, chosen_number):
    placement = list(np.array(item))
    print ("&&&&& selected exit ", placement , "  &&&&&&")
        
    ent_exitrate_list = []

    ##### per threshold 
    for thresh in entropy_threshold_list:
        print ("------------------ ", thresh, "------------------------")

        threshold_exit = []
        threshold_exitrate = [[] for i in range(branch_number)]

        ##### per sample
        for sample in range (sample_number):

            #### determining the exit branch based on entropy of the output and thresh
            for exit in range(branch_number):
                out = per_sample_out_vector_list[sample][exit]
                entropy = -1 * tf.math.reduce_sum((tf.math.log(out) * out)/ np.log(out.shape[1]))

                if (entropy < thresh or exit+1==branch_number):
                    threshold_exit.append(exit+1)
                    break


        # handling exit percentage part
        unique, counts = np.unique(threshold_exit, return_counts=True)
        exitper_list_dict = dict(zip(unique, counts))
        print ("exit rate per thresh ", exitper_list_dict)

        for i in range(branch_number):
            if (exitper_list_dict.get(placement[i]) is None):
                threshold_exitrate[i].append (0)
            else:
                threshold_exitrate[i].append(exitper_list_dict.get(placement[i]))


        ent_exitrate_list.append(np.mean(threshold_exitrate, axis=1)/sample_number)

    print("------------------------------------------------------------------")
    print("exit rate average ", np.mean(ent_exitrate_list, axis=0))
    break

%%%%%%%%%%%%%%%%%%%%%%% ResNet20 + CIFAR10(32x32)%%%%%%%%%%%%%%%%%%%%%%%%%%%%
computation latencies backbone(train data)    [1.06119835 1.56263262 1.35406359 1.35417639 1.75180006 1.41749927
 1.44090811 1.64432199 1.35139916 1.37869434]
computation latencies exitbranch(train data)    [0.57011324 0.57846146 0.58294255 0.57056284 0.57336909 0.564956
 0.56142836 0.56560254 0.56013978 0.55872234]
&&&&& selected exit  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   &&&&&&
------------------  1e-09 ------------------------
exit rate per thresh  {6: 14, 7: 23, 8: 40, 9: 123, 10: 49800}
------------------  0.052631527263157896 ------------------------
exit rate per thresh  {1: 802, 2: 3435, 3: 4150, 4: 2270, 5: 7682, 6: 7863, 7: 3649, 8: 4254, 9: 3258, 10: 12637}
------------------  0.10526305352631579 ------------------------
exit rate per thresh  {1: 1479, 2: 5763, 3: 5648, 4: 2715, 5: 8320, 6: 7636, 7: 3325, 8: 3619, 9: 2731, 10: 8764}
------------------  0.1578945797894737 ------------------------
exit 

In [6]:
##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches
print("%%%%%%%%%%%%%%%%%%%%%%% ResNet20 + CIFAR10(128x128)%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
per_sample_label_list = []
per_sample_out_vector_list = []
per_sample_comp_latency_backbone_list = []
per_sample_comp_latency_exitbranch_list = []


#### saving intermediate data once
for i in range (len(train_it)):
    temp_batch = train_it[i]

    for j in range (len(temp_batch[0])):
        pic = temp_batch[0][j]
        label = temp_batch[1][j]
        per_sample_label_list.append(np.array(label).reshape(1,10))

        res = model_CIFAR_ResNet(np.array(pic.reshape(1,image_resize,image_resize,3)), training = 1000)
   
        per_sample_out_vector_list.append(res[0])
        per_sample_comp_latency_backbone_list.append(res[1])
        per_sample_comp_latency_exitbranch_list.append(res[2])

print ("computation latencies backbone(train data)   ", np.mean(per_sample_comp_latency_backbone_list, axis=0)*1000)
print ("computation latencies exitbranch(train data)   ", np.mean(per_sample_comp_latency_exitbranch_list, axis=0)*1000)




##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches

entropy_threshold_list = np.linspace(0.000000001, 0.999999, num=20)

### all possible branches, and chosen number
chosen_number = branch_number
I = list(range(1, branch_number+1))

#### number of train samples for simulation
sample_number = train_it.samples



#### per placement, in this case it is only 1, all the possible branches
for item in combinations(I, chosen_number):
    placement = list(np.array(item))
    print ("&&&&& selected exit ", placement , "  &&&&&&")
        
    ent_exitrate_list = []

    ##### per threshold 
    for thresh in entropy_threshold_list:
        print ("------------------ ", thresh, "------------------------")

        threshold_exit = []
        threshold_exitrate = [[] for i in range(branch_number)]

        ##### per sample
        for sample in range (sample_number):

            #### determining the exit branch based on entropy of the output and thresh
            for exit in range(branch_number):
                out = per_sample_out_vector_list[sample][exit]
                entropy = -1 * tf.math.reduce_sum((tf.math.log(out) * out)/ np.log(out.shape[1]))

                if (entropy < thresh or exit+1==branch_number):
                    threshold_exit.append(exit+1)
                    break


        # handling exit percentage part
        unique, counts = np.unique(threshold_exit, return_counts=True)
        exitper_list_dict = dict(zip(unique, counts))
        print ("exit rate per thresh ", exitper_list_dict)

        for i in range(branch_number):
            if (exitper_list_dict.get(placement[i]) is None):
                threshold_exitrate[i].append (0)
            else:
                threshold_exitrate[i].append(exitper_list_dict.get(placement[i]))


        ent_exitrate_list.append(np.mean(threshold_exitrate, axis=1)/sample_number)

    print("------------------------------------------------------------------")
    print("exit rate average ", np.mean(ent_exitrate_list, axis=0))
    break

%%%%%%%%%%%%%%%%%%%%%%% ResNet20 + CIFAR10(128x128)%%%%%%%%%%%%%%%%%%%%%%%%%%%%
computation latencies backbone(train data)    [1.07179682 1.59485061 1.41119433 1.37436872 1.62892926 1.35556852
 1.34866416 1.57074742 1.33205153 1.32506626]
computation latencies exitbranch(train data)    [0.55097297 0.55931701 0.56123966 0.5415964  0.53471513 0.53009409
 0.52753703 0.53294127 0.53218225 0.52879668]
&&&&& selected exit  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   &&&&&&
------------------  1e-09 ------------------------
exit rate per thresh  {8: 7, 9: 133, 10: 49860}
------------------  0.052631527263157896 ------------------------
exit rate per thresh  {1: 2, 2: 1436, 3: 1498, 4: 1101, 5: 4443, 6: 4857, 7: 3187, 8: 8259, 9: 7077, 10: 18140}
------------------  0.10526305352631579 ------------------------
exit rate per thresh  {1: 8, 2: 2755, 3: 2718, 4: 1922, 5: 5653, 6: 5883, 7: 3525, 8: 8144, 9: 6246, 10: 13146}
------------------  0.1578945797894737 ------------------------
exit rate per thresh

In [6]:
##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches
print("%%%%%%%%%%%%%%%%%%%%%%% ResNet44 + CIFAR10(32x32)%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
per_sample_label_list = []
per_sample_out_vector_list = []
per_sample_comp_latency_backbone_list = []
per_sample_comp_latency_exitbranch_list = []


#### saving intermediate data once
for i in range (len(train_it)):
    temp_batch = train_it[i]

    for j in range (len(temp_batch[0])):
        pic = temp_batch[0][j]
        label = temp_batch[1][j]
        per_sample_label_list.append(np.array(label).reshape(1,10))

        res = model_CIFAR_ResNet(np.array(pic.reshape(1,image_resize,image_resize,3)), training = 1000)
   
        per_sample_out_vector_list.append(res[0])
        per_sample_comp_latency_backbone_list.append(res[1])
        per_sample_comp_latency_exitbranch_list.append(res[2])

print ("computation latencies backbone(train data)   ", np.mean(per_sample_comp_latency_backbone_list, axis=0)*1000)
print ("computation latencies exitbranch(train data)   ", np.mean(per_sample_comp_latency_exitbranch_list, axis=0)*1000)




##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches

entropy_threshold_list = np.linspace(0.000000001, 0.999999, num=20)

### all possible branches, and chosen number
chosen_number = branch_number
I = list(range(1, branch_number+1))

#### number of train samples for simulation
sample_number = train_it.samples



#### per placement, in this case it is only 1, all the possible branches
for item in combinations(I, chosen_number):
    placement = list(np.array(item))
    print ("&&&&& selected exit ", placement , "  &&&&&&")
        
    ent_exitrate_list = []

    ##### per threshold 
    for thresh in entropy_threshold_list:
        print ("------------------ ", thresh, "------------------------")

        threshold_exit = []
        threshold_exitrate = [[] for i in range(branch_number)]

        ##### per sample
        for sample in range (sample_number):

            #### determining the exit branch based on entropy of the output and thresh
            for exit in range(branch_number):
                out = per_sample_out_vector_list[sample][exit]
                entropy = -1 * tf.math.reduce_sum((tf.math.log(out) * out)/ np.log(out.shape[1]))

                if (entropy < thresh or exit+1==branch_number):
                    threshold_exit.append(exit+1)
                    break


        # handling exit percentage part
        unique, counts = np.unique(threshold_exit, return_counts=True)
        exitper_list_dict = dict(zip(unique, counts))
        print ("exit rate per thresh ", exitper_list_dict)

        for i in range(branch_number):
            if (exitper_list_dict.get(placement[i]) is None):
                threshold_exitrate[i].append (0)
            else:
                threshold_exitrate[i].append(exitper_list_dict.get(placement[i]))


        ent_exitrate_list.append(np.mean(threshold_exitrate, axis=1)/sample_number)

    print("------------------------------------------------------------------")
    print("exit rate average ", np.mean(ent_exitrate_list, axis=0))
    break

%%%%%%%%%%%%%%%%%%%%%%% ResNet44 + CIFAR10(32x32)%%%%%%%%%%%%%%%%%%%%%%%%%%%%
computation latencies backbone(train data)    [1.00379202 1.46268919 1.38500652 1.35506026 1.32993578 1.36724184
 1.39032877 1.48021005 1.66795323 1.36200067 1.32484176 1.32581272
 1.33411968 1.62183392 1.33781879 1.59764317 1.32429519 1.28260403
 1.28274433 1.29705244 1.32905243 1.3020076 ]
computation latencies exitbranch(train data)    [0.55167471 0.55648598 0.55566607 0.54259339 0.53706166 0.53820716
 0.5387153  0.53937181 0.54460309 0.5381208  0.53403195 0.53667655
 0.53930841 0.54038334 0.54030721 0.54171657 0.53493209 0.53356387
 0.53759444 0.54038672 0.54095125 0.54117776]
&&&&& selected exit  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]   &&&&&&
------------------  1e-09 ------------------------
exit rate per thresh  {8: 1, 10: 4, 11: 11, 12: 47, 13: 48, 14: 13, 15: 2, 16: 2, 17: 40, 18: 25, 19: 2, 20: 8, 22: 49797}
------------------  0.052631527263157896 ---------

In [6]:
##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches
print("%%%%%%%%%%%%%%%%%%%%%%% ResNet110 + CIFAR10(32x32)%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
per_sample_label_list = []
per_sample_out_vector_list = []
per_sample_comp_latency_backbone_list = []
per_sample_comp_latency_exitbranch_list = []


#### saving intermediate data once
for i in range (len(train_it)):
    temp_batch = train_it[i]

    for j in range (len(temp_batch[0])):
        pic = temp_batch[0][j]
        label = temp_batch[1][j]
        per_sample_label_list.append(np.array(label).reshape(1,10))

        res = model_CIFAR_ResNet(np.array(pic.reshape(1,image_resize,image_resize,3)), training = 1000)
   
        per_sample_out_vector_list.append(res[0])
        per_sample_comp_latency_backbone_list.append(res[1])
        per_sample_comp_latency_exitbranch_list.append(res[2])

print ("computation latencies backbone(train data)   ", np.mean(per_sample_comp_latency_backbone_list, axis=0)*1000)
print ("computation latencies exitbranch(train data)   ", np.mean(per_sample_comp_latency_exitbranch_list, axis=0)*1000)




##### passing TRAIN samples through model
##### this is to save computation time and exit rates for all the possible branches

entropy_threshold_list = np.linspace(0.000000001, 0.999999, num=20)

### all possible branches, and chosen number
chosen_number = branch_number
I = list(range(1, branch_number+1))

#### number of train samples for simulation
sample_number = train_it.samples



#### per placement, in this case it is only 1, all the possible branches
for item in combinations(I, chosen_number):
    placement = list(np.array(item))
    print ("&&&&& selected exit ", placement , "  &&&&&&")
        
    ent_exitrate_list = []

    ##### per threshold 
    for thresh in entropy_threshold_list:
        print ("------------------ ", thresh, "------------------------")

        threshold_exit = []
        threshold_exitrate = [[] for i in range(branch_number)]

        ##### per sample
        for sample in range (sample_number):

            #### determining the exit branch based on entropy of the output and thresh
            for exit in range(branch_number):
                out = per_sample_out_vector_list[sample][exit]
                entropy = -1 * tf.math.reduce_sum((tf.math.log(out) * out)/ np.log(out.shape[1]))

                if (entropy < thresh or exit+1==branch_number):
                    threshold_exit.append(exit+1)
                    break


        # handling exit percentage part
        unique, counts = np.unique(threshold_exit, return_counts=True)
        exitper_list_dict = dict(zip(unique, counts))
        print ("exit rate per thresh ", exitper_list_dict)

        for i in range(branch_number):
            if (exitper_list_dict.get(placement[i]) is None):
                threshold_exitrate[i].append (0)
            else:
                threshold_exitrate[i].append(exitper_list_dict.get(placement[i]))


        ent_exitrate_list.append(np.mean(threshold_exitrate, axis=1)/sample_number)

    print("------------------------------------------------------------------")
    print("exit rate average ", np.mean(ent_exitrate_list, axis=0))
    break

%%%%%%%%%%%%%%%%%%%%%%% ResNet110 + CIFAR10(32x32)%%%%%%%%%%%%%%%%%%%%%%%%%%%%
computation latencies backbone(train data)    [1.27908116 1.35492894 1.30633584 1.30529726 1.38806518 1.31773298
 1.32200212 1.32281848 1.32391569 1.32322505 1.32270598 1.32323905
 1.32360756 1.32472047 1.3233854  1.39121734 1.32444207 1.32335128
 1.32470798 1.6305693  1.32882537 1.34499961 1.31871814 1.32252617
 1.3328643  1.33493065 1.33015145 1.32807645 1.3267055  1.32853728
 1.33130296 1.40780528 1.51088671 1.43084661 1.33068219 1.32731767
 1.32729433 1.59393657 1.41442889 1.27291713 1.27760094 1.285372
 1.28934536 1.3104925  1.2908643  1.28820662 1.28926319 1.29195539
 1.37471923 1.45420452 1.28810961 1.29042663 1.28874732 1.28954314
 1.28951315]
computation latencies exitbranch(train data)    [0.5666563  0.53171494 0.52497724 0.52634132 0.53049194 0.53094642
 0.53183299 0.53182475 0.53253701 0.53192595 0.53085081 0.53233934
 0.53298107 0.53241436 0.53240119 0.533407   0.5331015  0.53127491
 0.53243231 

exit rate per thresh  {1: 19100, 2: 19756, 3: 5608, 4: 3138, 5: 1232, 6: 568, 7: 283, 8: 147, 9: 79, 10: 30, 11: 22, 12: 14, 13: 14, 14: 2, 15: 2, 16: 1, 17: 1, 19: 1, 21: 1, 24: 1}
------------------  0.7894728949473684 ------------------------
exit rate per thresh  {1: 26463, 2: 16759, 3: 3831, 4: 1906, 5: 591, 6: 263, 7: 103, 8: 51, 9: 15, 10: 7, 11: 4, 12: 6, 14: 1}
------------------  0.8421044212105263 ------------------------
exit rate per thresh  {1: 34948, 2: 11621, 3: 2248, 4: 895, 5: 189, 6: 64, 7: 20, 8: 9, 9: 3, 10: 2, 14: 1}
------------------  0.8947359474736842 ------------------------
exit rate per thresh  {1: 42395, 2: 6340, 3: 1015, 4: 216, 5: 28, 6: 6}
------------------  0.9473674737368422 ------------------------
exit rate per thresh  {1: 47807, 2: 2063, 3: 124, 4: 6}
------------------  0.999999 ------------------------
exit rate per thresh  {1: 50000}
------------------------------------------------------------------
exit rate average  [2.66084e-01 1.66710e-01 1

In [8]:
##### TEST PART, FOR SIMULATION ONLY

In [ ]:
##### passing TEST samples through model once and save the intermediate data
#### this is for doing the simulation

per_sample_label_list = []
per_sample_out_vector_list = []
per_sample_comp_latency_backbone_list = []
per_sample_comp_latency_exitbranch_list = []

for i in range (len(test_it)):
    temp_batch = test_it[i]

    for j in range (len(temp_batch[0])):
        pic = temp_batch[0][j]
        label = temp_batch[1][j]
        per_sample_label_list.append(np.array(label).reshape(1,10))

        res = model_CIFAR_ResNet(np.array(pic.reshape(1,image_resize,image_resize,3)), training = 1000)
   
        per_sample_out_vector_list.append(res[0])
        per_sample_comp_latency_backbone_list.append(res[1])
        per_sample_comp_latency_exitbranch_list.append(res[2])
        
print ("computation latencies backbone(test data)   ", np.mean(per_sample_comp_latency_backbone_list, axis=0)*1000)
print ("computation latencies exitbranch(test data)   ", np.mean(per_sample_comp_latency_exitbranch_list, axis=0)*1000)
